In [18]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string


In [30]:

labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]

batch_files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 
                'nogla0.csv','pew0.csv', 'rae0_0.csv', 
                 'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']

vid_files = ["foxnews_FULL.csv", "rae_FULL.csv",  "terror_FULL.csv",
             "usanews.csv" , "pew0.csv", "nogla0.csv", "drdisrespect_FULL.csv"]
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

files = []
files.extend(vid_files)
files.extend(genre_files)
print(files)

['foxnews_FULL.csv', 'rae_FULL.csv', 'terror_FULL.csv', 'usanews.csv', 'pew0.csv', 'nogla0.csv', 'drdisrespect_FULL.csv', 'gaming_channels_GENRE.csv', 'news_channel_GENRE.csv']


# TOP BI-GRAMS BASED ON TFIDF SCORES

In [35]:
src_folder = "tfidf_scores_WORDS_per_video/"
dest_folder = "tfidf_scores_WORDS_per_video/"
top_dfs = []
for file in files:
    vals = {}
    df = pd.read_csv(src_folder+file, index_col = 0)
    features = df.columns.difference(labels)
    dfs = []
    for label in labels:
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = "".join([w[0] for w in label.split()])
        if label == "Trolling":
            dl.index.name += "R"
        elif label == "Toxic":
            dl.index.name += "X"
        elif label == "Others":
            dl.index.name += "T"
        dl = dl.rename({dl.columns[0]:  dl.index.name + " value"}, axis='columns')
        dl.index.name += " words"
        dfs.append(dl)
    dfs = [d.reset_index(drop=False) for d in dfs]
    x = pd.concat(dfs, axis=1)
    display(x.style.set_caption(file))   
    x.to_csv(dest_folder+"TOP_WORDS_"+file)

,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,meltdown progress,2.309401,conservative tear,1.000000,beautiful kamala harris,0.447214,biden cheat,2.000000,traitor hang,1.506965,traitor hang,1.077060
1,magat meltdown,2.309401,fox total,1.000000,ivanka beautiful kamala,0.447214,gobble gobble,1.257449,civil war,0.860926,buck fiden,1.000000
2,magat meltdown progress,2.309401,sign traitor,1.000000,ivanka beautiful,0.447214,covid hoax,1.000000,need civil,0.601715,lose sucker,1.000000
3,maga tear,1.538530,jail right,1.000000,beautiful kamala,0.447214,hello dictatorship,1.000000,need civil war,0.601715,wait prison,1.000000
4,gobble gobble,1.517393,stimulus socialist,1.000000,kamala harris,0.447214,cheater bust,1.000000,hang gallow,0.595813,fk blm,1.000000
5,trump lol,1.474625,left garbage,1.000000,trump new,0.377964,raid server,1.000000,traitor hang treason,0.595813,fk acosta,1.000000
6,pardon turkey,1.425936,china virus,0.632233,big loser,0.377964,sign traitor,1.000000,hang treason,0.595813,pay mfer,1.000000
7,supreme court,1.302278,jail entire,0.577350,host big,0.377964,pay mfer,1.000000,traitor hang gallow,0.595813,swamp worm,1.000000
8,trump pardon,1.253666,entire left,0.577350,host big loser,0.377964,philadelphia rigger,1.000000,court traitor,0.475634,satan trump,1.000000
9,trump pardon turkey,1.084736,devil incarnate,0.577350,trump new host,0.377964,biden illuminati,1.000000,court traitor hang,0.475634,satan kuwait,1.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,bruh dude break,0.577350,00 hour,0.000000,toast thicc,0.447214,lesson old man,0.377964,00 hour,0.000000,00 hour,0.000000
1,bruh dude,0.577350,rae _raehype _raehype,0.000000,finally toast thicc,0.447214,teach lesson old,0.377964,rae _raehype _raehype,0.000000,rae _raehype _raehype,0.000000
2,dude break,0.577350,rae 2254,0.000000,toast thicc booty,0.447214,lesson old,0.377964,rae 2254,0.000000,rae 2254,0.000000
3,00 hour,0.000000,rae 2254 hug,0.000000,thicc booty,0.447214,time teach lesson,0.377964,rae 2254 hug,0.000000,rae 2254 hug,0.000000
4,rae _raehype,0.000000,rae 2nd,0.000000,finally toast,0.447214,time teach,0.377964,rae 2nd,0.000000,rae 2nd,0.000000
5,rae 2254 hug,0.000000,rae 2nd star,0.000000,rae 2nd,0.000000,teach lesson,0.377964,rae 2nd star,0.000000,rae 2nd star,0.000000
6,rae 2nd,0.000000,rae _rae100,0.000000,rae 2nd star,0.000000,old man,0.377964,rae _rae100,0.000000,rae _rae100,0.000000
7,rae 2nd star,0.000000,rae _raehype,0.000000,rae _rae100,0.000000,unlike video,0.301511,rae _raehype,0.000000,rae _raehype,0.000000
8,rae _rae100,0.000000,rae _raelove,0.000000,rae _raehype,0.000000,unsuscribe unlike,0.301511,rae _raelove,0.000000,rae _raelove,0.000000
9,rae _raelove,0.000000,pure support,0.000000,rae _raehype _raehype,0.000000,unlike video rae,0.301511,pure support,0.000000,pure support,0.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,die lmao,1.000000,black guy,2.852805,brian fan,1.000000,terroris simp,1.000000,kill nogla,1.000000,130 noob,1.000000
1,cory annoying,1.000000,guy nogla,1.340387,brian thicc,0.577350,dislike room,1.000000,burn wizard,1.000000,snitch fuck ridiculous,0.707107
2,lightning care,1.000000,guy nogla 2020,1.340387,brian thicc doe,0.577350,dislike trigger,1.000000,kill play,0.606875,snitch fuck,0.707107
3,sooo dirty,1.000000,nogla 2020,1.340387,thicc doe,0.577350,hope soon,0.577350,kill play guy,0.606875,snipe dickhead,0.627773
4,130 noob,1.000000,aussie weird,1.000000,release pleb,0.000000,trump2020 hope soon,0.577350,instant kill,0.577350,stream snipe dickhead,0.627773
5,breathes hes,0.707107,black guy nogla,0.927768,really sus,0.000000,trump2020 hope,0.577350,knife instant,0.577350,blocker fucker,0.622533
6,breathes hes impostor,0.707107,white guy nogla,0.480829,really think noah,0.000000,shoulda parent,0.447214,knife instant kill,0.577350,ad blocker fucker,0.622533
7,stream snipe dickhead,0.627773,white guy,0.422031,really want,0.000000,brown celebrity play,0.447214,play guy,0.513231,kill play,0.606875
8,snipe dickhead,0.627773,recently love terrorisor,0.000000,really want member,0.000000,orphan deserve shoulda,0.447214,leave open die,0.382034,kill play guy,0.606875
9,bad game watch,0.617562,recommendation midnight,0.000000,reason brian,0.000000,orphan deserve,0.447214,open die,0.382034,dang terroris,0.586032


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,biden suck,8.291875,china joe,12.082489,lady gag,3.000000,joe biden,9.277306,sunny yu,1.755135,biden suck,11.167500
1,joe biden,5.582116,vote trump,9.542417,lady gaga,1.918114,china joe,3.793307,beijing biden,1.574272,joe biden,7.951498
2,china joe,4.976935,biden suck,9.167500,commie bitch,1.000000,biden prison,3.663920,life matter,1.397841,lady gaga,5.049229
3,biden jail,4.881563,vote biden,7.403092,buck fiden,1.000000,biden china,3.000000,biden voter,1.111717,cnn suck,4.000000
4,sleepy joe,4.760010,biden prison,6.777448,make gag,0.899653,moron biden,2.852340,terror attack,1.076648,moron biden,3.469787
5,biden racist,4.519776,joe biden,5.888062,gaga make gag,0.899653,beijing biden,2.734768,trump hate,1.000000,biden supporter,3.447166
6,lady gaga,4.387135,china pay,5.086143,gag lady,0.899653,vote trump,2.712668,47 year,1.000000,biden harris,3.331422
7,biden prison,4.061779,china biden,4.716271,make gag lady,0.899653,biden corrupt,2.567092,chy na,1.000000,vote biden,3.328542
8,biden crime,3.905258,moron biden,4.704680,gaga make,0.873213,biden win,2.513682,trump china,1.000000,vote trump,2.848222
9,biden crime family,3.462861,biden 2020,4.682197,joe kameltoe,0.751828,fake news,2.454750,austria war,1.000000,china joe,2.808168


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,nancy cringe,1.000000,000 people,0.000000,felix gay,1.000000,members money,5.366563,000 people,0.000000,fake af,1.000000
1,nancy corpse,1.000000,response come,0.000000,require facetune picture,0.000000,charity members money,5.366563,response come,0.000000,try guy stfu,0.707107
2,nancy trash,1.000000,resize screen apparently,0.000000,resize screen apparently,0.000000,donations charity,5.366563,resize screen apparently,0.000000,try guy,0.707107
3,nancy boring,1.000000,respect btw,0.000000,respect btw,0.000000,donations charity members,5.366563,respect btw,0.000000,suck felix pick,0.577350
4,walk look like,0.424510,respect btw let,0.000000,respect btw let,0.000000,charity members,5.366563,respect btw let,0.000000,felix pick,0.577350
5,drunken man,0.424510,respond late,0.000000,respond late,0.000000,discord linked happened,0.638229,respond late,0.000000,suck felix,0.577350
6,way jack walk,0.424510,respond late everytime,0.000000,respond late everytime,0.000000,discord linked,0.638229,respond late everytime,0.000000,fck phasmo,0.333333
7,walk look,0.424510,respond ouija,0.000000,respond ouija,0.000000,join discord,0.430496,respond ouija,0.000000,fck phasmo audio,0.333333
8,way jack,0.393526,respond ouija board,0.000000,respond ouija board,0.000000,restart definitely,0.000000,respond ouija board,0.000000,lot time,0.333333
9,look like,0.352567,response come ask,0.000000,response come,0.000000,respond late,0.000000,response come ask,0.000000,audio lot time,0.333333


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,tom castillo,1.000000,blm joke,8.535057,sexy dick,1.000000,blm terrorist,1.237311,00 fucking,0.000000,pete fuckeddddd,1.000000
1,nogla racist,1.000000,death blm,3.503489,want dick,0.707107,covid fake,1.000000,speech hater,0.000000,shut bitch,1.000000
2,mom sus,1.000000,blm racist,3.000000,want dick jk,0.707107,purge 2021,1.000000,speech chat,0.000000,death blm,1.000000
3,speedy suck,1.000000,lives matter,1.705081,shaving ball hurt,0.585874,biden like,1.000000,speech chat fun,0.000000,code fckubtch,1.000000
4,mrtop5 gayyyyy,1.000000,george floyd,1.368968,shaving ball,0.585874,warming myth,0.722942,speech damit,0.000000,sexy dick,1.000000
5,richie 21 gay,0.707107,blm suck,1.322003,want suck,0.577350,voting joe know,0.707107,speech die,0.000000,chat retard,1.000000
6,richie 21,0.707107,blm terrorist,1.000000,want suck dick,0.577350,trump gae biden,0.707107,speech die shut,0.000000,white boy,0.828362
7,pete fuckin,0.707107,poc blm,1.000000,suck dick,0.577350,voting joe,0.707107,speech free,0.000000,pete fuckin idiot,0.707107
8,pete fuckin idiot,0.707107,hate jews,1.000000,ban nogla,0.559914,eat kid,0.707107,speech free spam,0.000000,pete fuckin,0.707107
9,deluxe fuck,0.600376,osers arginalized,1.000000,dick live,0.529994,eat kid peel,0.707107,speech hater block,0.000000,shut blm fucker,0.707107


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,people say member,2.309401,ni er,1.000000,baby meet,0.447214,000 000,0.000000,000 000,0.000000,non members,1.000000
1,people say,2.309401,000 000,0.000000,baby meet bedroom,0.447214,nice chat 3k,0.000000,nice chat 3k,0.000000,ew public chat,0.577350
2,say member,2.309401,nice chat 3k,0.000000,meet bedroom,0.447214,nice car,0.000000,nice car,0.000000,public chat,0.577350
3,member chat,1.374228,nice car,0.000000,hey baby,0.447214,nice 100 floor,0.000000,nice 100 floor,0.000000,ew public,0.577350
4,anti member,1.188723,nice 100 floor,0.000000,hey baby meet,0.447214,nice 100,0.000000,nice 100,0.000000,_flappin _flappin,0.475917
5,anti member chat,1.188723,nice 100,0.000000,000 000,0.000000,ni er,0.000000,ni er,0.000000,start play trash,0.336108
6,non members,1.000000,news chance free,0.000000,new song drop,0.000000,news chance free,0.000000,news chance free,0.000000,live 22 minute,0.336108
7,doc boomer,1.000000,news chance,0.000000,nice 100 floor,0.000000,news chance,0.000000,news chance,0.000000,live 22,0.336108
8,want member,0.611136,new transition doc,0.000000,nice 100,0.000000,new transition doc,0.000000,new transition doc,0.000000,22 minute,0.336108
9,bro bros trash,0.600742,new transition,0.000000,ni er,0.000000,new transition,0.000000,new transition,0.000000,minute start,0.336108


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,people say member,2.372892,blm joke,8.104467,ban nogla,1.000000,members money,5.366563,play guy,1.000000,white boy,1.566457
1,say member,2.351603,death blm,4.000000,live matter,1.000000,donations charity,5.366563,kill nogla,1.000000,stream snipe,1.000000
2,people say,2.199852,black guy,3.505535,felix gay,1.000000,charity members,5.366563,guy lol,1.000000,public chat,1.000000
3,member chat,1.264009,blm racist,3.000000,meet bedroom,1.000000,charity members money,5.366563,00 rae,0.000000,chat retard,1.000000
4,anti member chat,1.236869,lives matter,2.342428,00 rae,0.000000,donations charity members,5.366563,potato powers activate,0.000000,die lol,1.000000
5,anti member,1.236869,blm suck,2.000000,potato pick,0.000000,blm terrorist,1.483568,potato noise,0.000000,blm suck,1.000000
6,suck bad,1.000000,death black,1.489961,potato potato,0.000000,purge 2021,1.000000,potato pick,0.000000,non members,1.000000
7,bro bros,1.000000,george floyd,1.457013,potato potato family,0.000000,biden like,1.000000,potato potato,0.000000,ad blocker,1.000000
8,tom castillo,1.000000,guy nogla 2020,1.443434,potato potato potato,0.000000,ban nogla,1.000000,potato potato family,0.000000,death blm,1.000000
9,smell like,1.000000,guy nogla,1.411289,potato powers,0.000000,join discord,1.000000,potato potato potato,0.000000,guy stfu,1.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,biden suck,8.291081,china joe,12.101228,lady gag,3.000000,joe biden,10.618480,traitor hang,3.749889,biden suck,11.164323
1,joe biden,7.532736,vote trump,9.403320,lady gaga,1.919480,biden prison,3.807546,civil war,2.000000,joe biden,7.898715
2,sleepy joe,5.903194,biden suck,9.164323,make gag,1.007282,china joe,3.801986,sunny yu,1.754881,lady gaga,5.050634
3,china joe,4.986737,vote biden,7.472942,gag lady,1.007282,vote trump,3.723379,beijing biden,1.574307,cnn suck,4.000000
4,biden jail,4.882732,biden prison,6.890406,make gag lady,1.007282,biden win,3.014630,biden voter,1.465081,vote biden,3.875534
5,lady gaga,4.371027,joe biden,6.149795,commie bitch,1.000000,biden china,3.000000,life matter,1.398097,biden harris,3.518930
6,biden racist,4.315291,china pay,5.086512,kamala harris,1.000000,fake news,2.942018,terror attack,1.077138,moron biden,3.477117
7,biden prison,4.063162,china virus,4.946780,trump new,1.000000,moron biden,2.857837,mr president,1.000000,biden supporter,3.350534
8,biden crime,3.780495,biden 2020,4.941256,buck fiden,1.000000,biden harris,2.829684,say biden,1.000000,vote trump,2.965617
9,crime family,3.464373,china biden,4.716686,gaga make,0.977828,vote biden,2.787209,biden obama,1.000000,china joe,2.813161


# TOP BI-GRAMS BASED ON LOWEST P-VALUE

In [89]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder = "corr_words_per_label/"
corr_dfs = []
for file in files:
    df = pd.read_csv(src_folder+file, index_col = 0) 
    df.dropna(axis = 0, how = 'all', inplace = True)
    
    dic = {}
    dfs = {}
    #corr coeffcorr coeff
    for label in labels:
        dp = df[(df[label + " p-val"] > 0 )& ( df[label + " corr coeff"] > 0)].sort_values(by=[label + " p-val"], ascending = True).head(10)
        top_features = list(dp[label + " p-val"].index)
        #display(df[label + " p-val"])
        #dt = dp[ [label + " p-val", label + " corr coeff"]].T[top_features]
        dic[label] = top_features
   
    
    corr_coeff = {}
    for label in labels:
        col = ["top " + label]
        col.extend(dic[label])
        corr_df = pd.DataFrame(columns= col )
        
        for row in labels:
            new_row = {}
            for word in dic[label]:
                x = df.loc[word, row + " corr coeff"]
                row_label = "".join([w[0] for w in row.split()])
                
                if row == "Trolling":
                    row_label += "R"
                elif row == "Toxic":
                    row_label += "X"
                elif row == "Others":
                    row_label += "T" 
                
                new_row["top " + label] = row_label
                new_row[word] = x
            corr_df = corr_df.append(new_row, ignore_index=True)
        
        display(corr_df.set_index("top " + label).T.style.set_caption(file))
    

top Direct Harassment,DH,HS,SH,TR,OT,TX
bye bye,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
fox news,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
biden win,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
wear mask,0.774597,-0.309839,-0.309839,-0.309839,0.464758,-0.309839
president elect,0.774597,0.464758,-0.309839,-0.309839,-0.309839,-0.309839
vote biden,0.707107,-0.424264,-0.424264,0.282843,-0.424264,0.282843
supreme court,0.695608,-0.417365,-0.417365,0.139122,0.417365,-0.417365
trump pardon,0.695608,-0.417365,-0.417365,0.417365,-0.417365,0.139122
pardon trump,0.695608,-0.417365,-0.417365,0.417365,-0.417365,0.139122
pardon turkey,0.695608,-0.417365,-0.417365,0.417365,-0.417365,0.139122


top Hate Speech,DH,HS,SH,TR,OT,TX
country communist,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
dead people,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
dead people vote,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
lose lawsuit katie,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228
manchurian candidate,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228
manchurian candidate represent,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228
mask course,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228
mask course certify,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228
mr president,-0.316228,0.632456,-0.316228,-0.316228,0.632456,-0.316228
news ppl,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228


top Sexual Harassment,DH,HS,SH,TR,OT,TX
beautiful kamala,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
beautiful kamala harris,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
big loser,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
host big,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
host big loser,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
ivanka beautiful,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
ivanka beautiful kamala,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
kamala harris,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
new host,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
new host big,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228


top Trolling,DH,HS,SH,TR,OT,TX
biden cheat,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
biden voter,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
talk gobbles,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
pardon turkey pardon,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
pardon trump pardon,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
stock market,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
pardon stock market,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
family joke,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
family joke funny,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
trump pardon gobble,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122


top Others,DH,HS,SH,TR,OT,TX
traitor hang,0.282843,-0.424264,-0.424264,-0.424264,0.707107,0.282843
civil war,-0.417365,0.139122,-0.417365,0.417365,0.695608,-0.417365
sock presses mouth,0.632456,-0.316228,-0.316228,-0.316228,0.632456,-0.316228
war begin,-0.316228,-0.316228,-0.316228,0.632456,0.632456,-0.316228
mr president,-0.316228,0.632456,-0.316228,-0.316228,0.632456,-0.316228
life mr president,-0.316228,0.632456,-0.316228,-0.316228,0.632456,-0.316228
life mr,-0.316228,0.632456,-0.316228,-0.316228,0.632456,-0.316228
let second civil,-0.316228,-0.316228,-0.316228,0.632456,0.632456,-0.316228
let second,-0.316228,-0.316228,-0.316228,0.632456,0.632456,-0.316228
second civil,-0.316228,-0.316228,-0.316228,0.632456,0.632456,-0.316228


...

top Hate Speech,DH,HS,SH,TR,OT,TX
kick joe,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
safe america,-0.309839,0.774597,-0.309839,-0.309839,-0.309839,0.464758
ruin california vote,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
ruin california,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
indianapolis countyworkers citizen,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
china ccp owns,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
china joe tape,0.464758,0.774597,-0.309839,-0.309839,-0.309839,-0.309839
rid commie,-0.309839,0.774597,-0.309839,-0.309839,-0.309839,0.464758
china pay,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
receptionist say know,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839


top Sexual Harassment,DH,HS,SH,TR,OT,TX
make gag,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
gag lady,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
make gag lady,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
gaga make,0.417365,-0.417365,0.695608,-0.417365,-0.417365,0.139122
hell kameltoe,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
lady gaga man,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
lady gaga lady,-0.316228,-0.316228,0.632456,-0.316228,-0.316228,0.632456
joe kameltoe,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
joe hell kameltoe,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
trump new,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228


top Trolling,DH,HS,SH,TR,OT,TX
blm biden laptop,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
socialism communism,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
steal election,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
biden run,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
biden love,-0.309839,-0.309839,-0.309839,0.774597,-0.309839,0.464758
terrorist attack,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
biden laptop matter,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
biden laptop,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
biden vote satan,-0.309839,-0.309839,-0.309839,0.774597,-0.309839,0.464758
biden know,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839


top Others,DH,HS,SH,TR,OT,TX
know talk,0.464758,-0.309839,-0.309839,-0.309839,0.774597,-0.309839
terror attack,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
dem want,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
attack vienna,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
attack austria,-0.424264,0.282843,-0.424264,0.282843,0.707107,-0.424264
biden obama,-0.424264,0.282843,-0.424264,0.282843,0.707107,-0.424264
traitor hang,0.269680,-0.134840,-0.539360,-0.539360,0.674200,0.269680
civil war,-0.539360,0.404520,-0.539360,0.000000,0.674200,0.000000
lock lock,-0.134840,0.269680,-0.539360,0.269680,0.674200,-0.539360
holy spirit,-0.316228,0.632456,-0.316228,-0.316228,0.632456,-0.316228


top Toxic,DH,HS,SH,TR,OT,TX
žðÿ žðÿ žðÿ,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
bin biden crime,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
žðÿ žðÿ,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kamala black,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kamala black dementia,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
dementia joe forgot,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kkk hunter,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kkk hunter crack,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
vote satanist,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
vote satanist pedophiles,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597


In [ ]:
        corr_df = pd.DataFrame(columns = ['corr coeff', 'p-value'])
        dp = df[(df[label + " "] > 0)].sort_values(by=[label], ascending = True).head(10)
        top_features = list(dp[label].index)
        #print(dp[label])
        ds = df_2[label].T[top_features]
        #display(ds.T[top_features])
       
        corr_df['words'] = dp.index
        corr_df['p-value'] = list(dp[label])
        corr_df['corr coeff']  = list(ds)
        corr_df = corr_df.set_index('words')
        corr_df.index.name = fn + " - " + label
        display(corr_df)
        corr_df.to_csv(dest_folder+"CORR_ANALYSIS_"+file)
        

In [40]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder =  "tfidf_scores_POS_per_video/"

for file in genre_files:
    df = pd.read_csv(src_folder+file, index_col = 0) 
    features = df.columns.difference(labels)
    for label in labels: 
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = file
        dl = dl.rename({dl.columns[0]: label}, axis='columns')
        
        display(dl)


,Direct Harassment
gaming_channels_GENRE.csv,
noun,5.932765
adj,5.304345
propn,4.375266
verb,3.595740
pron,3.011161
det,2.660375
aux,2.203011
det noun,2.009084
adj noun,1.897509


,Hate Speech
gaming_channels_GENRE.csv,
noun,33.668220
propn verb,27.484534
propn,20.021959
verb,18.332486
propn aux,5.975343
aux,4.283330
propn aux det noun,4.265662
noun noun,4.117543
propn aux det,3.887602


,Sexual Harassment
gaming_channels_GENRE.csv,
noun,1.829969
verb,1.301574
propn,1.190427
adj,1.015586
pron,0.907469
adj propn,0.868857
propn verb noun,0.708981
propn verb,0.655255
adp noun noun propn,0.613291


,Trolling
gaming_channels_GENRE.csv,
propn,4.380458
noun,3.728319
verb,3.490139
aux,3.217730
punct,3.142331
noun punct,2.980001
pron,2.825418
adv det propn noun,2.798669
punct adv det propn,2.798669


,Others
gaming_channels_GENRE.csv,
verb,0.981996
verb propn,0.855102
verb det propn,0.658518
adp det noun,0.636822
det,0.601597
propn,0.586069
adp det,0.581903
det propn,0.510162
adp,0.469519


...

,Hate Speech
news_channel_GENRE.csv,
propn,88.870788
noun,80.992302
punct,58.195538
verb,50.482515
adp,46.099317
propn propn,38.031241
det,36.809460
aux,32.976515
emoji,32.881169


,Sexual Harassment
news_channel_GENRE.csv,
propn,3.195031
noun,2.269076
verb,2.003061
propn propn,1.933734
det,1.604342
punct,1.587059
pron,1.462102
punct punct,1.451485
propn verb,1.344852


,Trolling
news_channel_GENRE.csv,
propn,56.056145
noun,45.845107
verb,36.601344
punct,34.140985
propn propn,26.214219
adp,25.786085
aux,22.724393
det,22.264011
adj,17.683020


,Others
news_channel_GENRE.csv,
propn,8.910536
noun,7.566555
verb,5.984656
punct,5.852794
det,5.235247
adp,4.734272
pron,4.188924
det noun,3.754812
aux,3.699306


,Toxic
news_channel_GENRE.csv,
propn,52.543534
noun,50.549503
verb,32.217801
punct,28.924005
propn propn,22.642132
adp,21.683983
det,19.829922
aux,16.542350
emoji,16.095242
